# NLP Syntactic analysis

The Syntactic analysis in NLP deals with the correct formation of the text. Usually deals with the concept of parsing study at the level of how sentences are formed with respect to a grammar and it's structure. This level of linguistic processing utilizes a parsers to extract the structure fo the phrases.

## Context Free Grammars
Context free grammars are a powerful generation tool which allow us to represent a wide variety of free ontext languages which are impossible to express using regular expressions

Let's define a simple grammar to represent the language : <img src="https://latex.codecogs.com/gif.latex?O^n1^n" />

In [3]:
import nltk
from nltk.draw.tree import draw_trees

grammar = nltk.CFG.fromstring("""
S -> Z A | eps 
A -> B O | '1'
B -> Z A
Z -> '0'
O -> '1'
""")

Let's process a string

In [4]:
parser = nltk.ChartParser(grammar)
phrase = ['0', '0', '0', '1', '1', '1']
for tree in parser.parse(phrase):
    print(tree)
    draw_trees(tree)

(S (Z 0) (A (B (Z 0) (A (B (Z 0) (A 1)) (O 1))) (O 1)))


**Exercise:** Design a grammar to recognize the language <img src="https://latex.codecogs.com/gif.latex?${a^{i}b^{j}c^{k} | i, j, k \geq 0, i = k$" /> test it using the strings aaabbbc, aaabccc and aabbbbccc 

In [1]:
#Your code here

## Pushdown automata
A pushdown automata is a theoreticla machine which can model free context grammars. It's matematical definition is confomed bya 6 tuple which includes the following elements:
* Q: Set of the states of the automata
* Sigma: Set of input symbols
* Gamma: set of stack symbols
* Delta: transition function
* F: set of acceptance states


Let's clone the repository

In [1]:
!git clone https://github.com/MaxSob/SimplePushdownAutomata.git

Clonando en 'SimplePushdownAutomata'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 8), reused 16 (delta 6), pack-reused 0
Desempaquetando objetos: 100% (22/22), listo.


Let's build the grammar

In [3]:
!more ./SimplePushdownAutomata/test.py

from PushdownAutomata import PushdownAutomata

#Automata definition
q = ['q1','q2','q3','q4']
sigma = ['0','1']
gamma = ['0',"$"]
delta = {}
f = ['q1','q4']

#Rule definition
a = PushdownAutomata(q, sigma, gamma, delta, 'q1', f)
a.addRule('q1','q2','epsilon','epsilon','$')
a.addRule('q2','q2','0', 'epsilon', '0')
a.addRule('q2','q3','1', '0', 'epsilon')
a.addRule('q3','q3','1', '0', 'epsilon')
a.addRule('q3','q4','epsilon', '$', 'epsilon')

#Process a String
accepted_paths = a.processInput('000111')
a.printPaths(accepted_paths)


Let's run the automata

In [4]:
!python ./SimplePushdownAutomata/test.py

[[(None, 'q1', [])]]
******************************
Processing char 1 : 0
q1 [] 0
Applying q1: (epsilon,epsilon)-->$ : q2
Applying q2: (0,epsilon)-->0 : q2
******************************
Processing char 2 : 0
q2 ['$', '0'] 0
Applying q2: (0,epsilon)-->0 : q2
******************************
Processing char 3 : 0
q2 ['$', '0', '0'] 0
Applying q2: (0,epsilon)-->0 : q2
******************************
Processing char 4 : 1
q2 ['$', '0', '0', '0'] 1
Applying q2: (1,0)-->epsilon : q3
******************************
Processing char 5 : 1
q3 ['$', '0', '0'] 1
Applying q3: (1,0)-->epsilon : q3
******************************
Processing char 6 : 1
q3 ['$', '0'] 1
Applying q3: (1,0)-->epsilon : q3
Applying q3: (epsilon,$)-->epsilon : q4
Printing path: 1
None
q1: (epsilon,epsilon)-->$ : q2
q2: (0,epsilon)-->0 : q2
q2: (0,epsilon)-->0 : q2
q2: (0,epsilon)-->0 : q2
q2: (1,0)-->epsilon : q3
q3: (1,0)-->epsilon : q3
q3: (1,0)-->epsilon : q3
q3: (epsilon,$)-->epsilon : q4


## CYK algorithm
**Exercise:** implement a simple CYK algorithm like https://github.com/RobMcH/CYK-Parser/blob/master/cyk_parser.py

In [6]:
def CYK(rules, string):
    table = []
    #Your code here
    return table

## Parsers

Let's see another example with an ambiguous grammar:

In [5]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

This grammar permits the sentence to be analyzed in two ways, depending on whether the prepositional phrase in my pajamas describes the elephant or the shooting event.

![title](resources/NLP_04_syntactic_analysis_tree_01.png)

![title](resources/NLP_04_syntactic_analysis_tree_02.png)

In [6]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print(tree)
    draw_trees(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## Recursive descent parser

The simplest kind of parser interprets a grammar as a specification of how to break a high-level goal into several lower-level subgoals. The top-level goal is to find an S. The S → NP VP production permits the parser to replace this goal with two subgoals: find an NP, then find a VP. Each of these subgoals can be replaced in turn by sub-sub-goals, using productions that have NP and VP on their left-hand side. Eventually, this expansion process leads to subgoals such as: find the word telescope. Such subgoals can be directly compared against the input sequence, and succeed if the next word is matched. If there is no match the parser must back up and try a different alternative.

In [7]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

Parsing revursively

In [8]:
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)
    draw_trees(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [10]:
nltk.app.rdparser()

Recursive descent parsing has three key shortcomings. First, left-recursive productions like NP -> NP PP send it into an infinite loop. Second, the parser wastes a lot of time considering words and structures that do not correspond to the input sentence. Third, the backtracking process may discard parsed constituents that will need to be rebuilt again later. For example, backtracking over VP -> V NP will discard the subtree created for the NP. If the parser then proceeds with VP -> V NP PP, then the NP subtree must be created all over again.

## Shift reduce parser
A simple kind of bottom-up parser is the shift-reduce parser. In common with all bottom-up parsers, a shift-reduce parser tries to find sequences of words and phrases that correspond to the right hand side of a grammar production, and replace them with the left-hand side, until the whole sentence is reduced to an S.

The shift-reduce parser repeatedly pushes the next input word onto a stack; this is the shift operation. If the top n items on the stack match the n items on the right hand side of some production, then they are all popped off the stack, and the item on the left-hand side of the production is pushed on the stack. This replacement of the top n items with a single item is the reduce operation. This operation may only be applied to the top of the stack; reducing items lower in the stack must be done before later items are pushed onto the stack. The parser finishes when all the input is consumed and there is only one item remaining on the stack, a parse tree with an S node as its root. The shift-reduce parser builds a parse tree during the above process. Each time it pops n items off the stack it combines them into a partial parse tree, and pushes this back on the stack. We can see the shift-reduce parsing algorithm in action using the graphical demonstration 

In [11]:
import nltk
nltk.app.srparser()

Using the shift reduce parser

In [6]:
sr_parser = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)
    draw_trees(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


A shift-reduce parser can reach a dead end and fail to find any parse, even if the input sentence is well-formed according to the grammar. When this happens, no input remains, and the stack contains items which cannot be reduced to an S. The problem arises because there are choices made earlier that cannot be undone by the parser (although users of the graphical demonstration can undo their choices). There are two kinds of choices to be made by the parser: (a) which reduction to do when more than one is possible (b) whether to shift or reduce when either action is possible.

The advantage of shift-reduce parsers over recursive descent parsers is that they only build structure that corresponds to the words in the input. Furthermore, they only build each sub-structure once, e.g.  NP(Det(the), N(man)) is only built and pushed onto the stack a single time, regardless of whether it will later be used by the VP -> V NP PP reduction or the NP -> NP PP reduction.